# Parameterisation

## Overview
**Questions**
- How do I get the parameters for a potential?
- Why is this a limitation of molecular dynamics?

**Objectives**
- Emphasize the requirement to use or develop accurate potential models for the system under study.

We now understand that there are a series of different models that can be used to simulate atoms. 
These depend on the type of interaction (bonded/non-bonded, angle/dihedral, etc.).
However, their nature also depends of the underlaying chemistry of the atoms. 
For example, with the Lennard-Jones interaction discussed in the previous lesson, the shape of the function (e.g. the depth and position of the potential energy well) is specific to the interaction between two atoms. 
Therefore, it is necessary that we understand *how* these functions are determined. 
This is called **force-field parameterisation**.

## How to parameterise a forcefield?

The purpose of force-field parameterisation is to create a model that is capable of accurately reproducing the energy or forces in an interaction. 
This means that the parameters should really be obtained from a more accurate technique than classical simulation. 
Commonly this means either experimental measurements, e.g. X-ray crystallography, or quantum mechanical calculations, we will be focusing on the latter. 

Mroe can be found out about quantum mechanical calculations in the textbooks mentioned in the introduction (in particular Jeremy Harvey's Computational Chemistry Primer [[1]](#References)).
However, for our current purposes all that is needed to be known is that quantum mechanical calculations are more accurate than classical simulation (as they account for the electronic structure of the atoms) but they are significantly more limited to the number of atoms that they can compute (as they account for the electronic structure of the atoms). 

## Key Points
- Classical methods simplify the atomistic modelling by treating the atoms using mathematical functions
- There are a wide range of different mathematical functions available

# References

1. J. Harvey, (2017). *Computational Chemistry*. Oxford, UK. Oxford University Press